<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c85ac57324aa467b478dfe9e2f7eaaef994e3d8362b0a81755dc094102e9dccd
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 10:19:01
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: -1.50 L (-1.07%)
Current PnL: -22.65 L (-15.09%)
CY Booked + Current PnL: -10.75 L (-7.16%)
-------------------
Total profit:  1.29 L
Total loss:  -23.94 L
-------------------
Total Booked + Current PnL: 16.23 L (13.18%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.57%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.86 L (64.72%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.47,54.0,X-LC,3.73,211912.0,7412.0,15893.0,-1.07,3.62,7.50,11.39,10.0,0.47,1.54,12.90,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,94.24,42.0,M-SC,1.36,83644.0,-17133.0,17231.0,0.17,-17.00,20.60,0.09,245.0,-0.99,0.61,9.89,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.94,52.0,X-MC,1.01,207720.0,15978.0,21790.0,0.46,8.33,10.49,19.69,99.0,0.73,1.51,15.70,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,48.0,X-LC,2.85,214765.0,4399.0,24376.0,0.12,2.09,11.35,13.68,37.0,0.18,1.56,18.96,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,39.0,X-LC,2.75,194500.0,-5638.0,26530.0,-0.38,-2.82,13.64,10.44,4.0,-0.21,1.41,2.59,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,38.11,71.0,M-LC,8.70,225395.0,20319.0,92254.0,2.49,9.91,40.93,54.89,52.0,0.22,1.64,36.39,X5K,ATH,METALS
4,ANGELONE,3033.00,10.71,63.0,X-SC,10.32,195857.0,4851.0,52842.0,2.04,2.54,26.98,30.21,157.0,0.09,1.42,23.95,X40N,NTT,FINANCE
66,SONACOMS,1006.00,-37.62,60.0,M-SC,12.95,77700.0,-23459.0,98353.0,1.89,-23.19,126.58,74.03,202.0,-0.24,0.56,9.56,AR,BTT,AUTO
34,ICICIPRULI,790.00,-20.88,51.0,X-MC,1.93,136063.0,275.0,42479.0,1.42,0.20,31.22,31.48,107.0,0.01,0.99,12.35,X40,ATH,INSURANCE
47,LTIM,7230.20,1.16,69.0,H-LC,3.11,250898.0,4747.0,74467.0,1.41,1.93,29.68,32.18,16.0,0.06,1.82,38.67,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-55.15,11.0,X-LC,1.12,142369.0,-134154.0,244234.0,-40.64,-48.51,171.55,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
53,RAJOOENG,143.1,-42.97,41.0,H-SC,17.38,86500.0,-16000.0,56597.0,-3.32,-15.61,65.43,39.61,114.0,-0.28,0.63,5.64,AR,ATH,MISC
10,BANDHANBNK,400.0,-11.95,49.0,H-SC,2.98,219733.0,-58826.0,316262.0,-2.67,-21.12,143.93,92.42,151.0,-0.19,1.60,26.99,XY24,NTT,BANKS
68,SURYODAY,240.0,60.09,56.0,H-SC,10.52,147002.0,-32069.0,97800.0,-2.21,-17.91,66.53,36.71,135.0,-0.33,1.07,45.86,XR,NTT,BANKS
81,VALIANTORG,838.0,-282.78,25.0,H-SC,3.03,101660.0,-61945.0,183262.0,-2.13,-37.86,180.27,74.15,139.0,-0.34,0.74,30.31,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.0,43.54,49.0,M-MC,12.6,226805.0,1843.0,165817.0,-0.60,0.82,73.11,74.53,192.0,0.01,1.65,33.34,XY24,BTT,STEEL
54,RECLTD,446.0,45.36,43.0,M-LC,6.2,203665.0,495.0,41629.0,-0.22,0.24,20.44,20.74,55.0,0.01,1.48,5.80,XY25,NTT,FINANCE
85,WIPRO,420.0,-13.34,49.0,M-LC,6.0,152142.0,1197.0,108264.0,0.11,0.79,71.16,72.51,53.0,0.01,1.10,6.86,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,106.53,61.0,H-SC,11.62,130734.0,-11745.0,31272.0,-0.66,-8.24,23.92,13.70,163.0,-0.38,0.95,54.30,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,94.24,42.0,M-SC,1.36,83644.0,-17133.0,17231.0,0.17,-17.00,20.60,0.09,245.0,-0.99,0.61,9.89,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,49.0,H-SC,2.07,221211.0,-48456.0,84790.0,0.04,-17.97,38.33,13.47,138.0,-0.57,1.61,12.48,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,1.97,139909.0,-35994.0,78013.0,1.18,-20.46,55.76,23.89,149.0,-0.46,1.02,20.83,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.22,41.0,H-MC,7.03,101175.0,-30660.0,72886.0,-0.50,-23.26,72.04,32.03,98.0,-0.42,0.73,13.32,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203665.0,495.0,41629.0,-0.22,0.24,20.44,20.74,55.0,0.01,1.48,5.80,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,49.0,M-MC,12.60,226805.0,1843.0,165817.0,-0.60,0.82,73.11,74.53,192.0,0.01,1.65,33.34,XY24,BTT,STEEL
31,HINDZINC,730.22,38.11,71.0,M-LC,8.70,225395.0,20319.0,92254.0,2.49,9.91,40.93,54.89,52.0,0.22,1.64,36.39,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,54.0,M-LC,9.55,157258.0,16418.0,28071.0,-1.14,11.66,17.85,31.59,66.0,0.58,1.14,41.23,XY24,NTT,BANKS
85,WIPRO,420.00,-13.34,49.0,M-LC,6.00,152142.0,1197.0,108264.0,0.11,0.79,71.16,72.51,53.0,0.01,1.10,6.86,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,38.11,71.0,M-LC,8.70,225395.0,20319.0,92254.0,2.49,9.91,40.93,54.89,52.0,0.22,1.64,36.39,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,62.0,H-MC,12.71,192819.0,13011.0,99398.0,-1.17,7.24,51.55,62.52,88.0,0.13,1.40,43.79,XR,NTT,BANKS
36,INDIAMART,4810.62,-55.20,39.0,H-SC,8.32,125334.0,1998.0,129633.0,-1.41,1.62,103.43,106.72,119.0,0.02,0.91,25.38,AR,ATH,MISC
85,WIPRO,420.00,-13.34,49.0,M-LC,6.00,152142.0,1197.0,108264.0,0.11,0.79,71.16,72.51,53.0,0.01,1.10,6.86,XR,NTT,IT
84,WHIRLPOOL,2270.00,-48.37,32.0,M-SC,3.38,87368.0,-4130.0,82886.0,0.06,-4.51,94.87,86.07,223.0,-0.05,0.63,26.87,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,11.0,X-LC,1.12,142369.0,-134154.0,244234.0,-40.64,-48.51,171.55,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,31.0,X-SC,37.83,51794.0,-13212.0,164140.0,-0.84,-20.32,316.91,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
5,ASIANPAINT,3465.66,-18.68,33.0,X-LC,5.32,205401.0,-46367.0,99578.0,-0.36,-18.42,48.48,21.13,27.0,-0.47,1.49,10.48,X40,ATH,PAINTS
9,BAJAJHFL,181.50,-15.54,33.0,X-MC,5.47,177792.0,-22714.0,119867.0,-0.29,-11.33,67.42,48.45,90.0,-0.19,1.29,2.53,X40N,ATH,FINANCE
28,HAVELLS,2069.16,-7.31,33.0,X-MC,0.90,229804.0,-17927.0,92979.0,-1.52,-7.24,40.46,30.30,92.0,-0.19,1.67,5.09,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,HAVELLS,2069.16,-7.31,33.0,X-MC,0.90,229804.0,-17927.0,92979.0,-1.52,-7.24,40.46,30.30,92.0,-0.19,1.67,5.09,X40,ATH,ELECTRICAL
64,SIEMENS,4671.50,-3.90,41.0,H-LC,0.92,154570.0,-31525.0,79001.0,0.45,-16.94,51.11,25.51,15.0,-0.40,1.12,13.91,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.94,52.0,X-MC,1.01,207720.0,15978.0,21790.0,0.46,8.33,10.49,19.69,99.0,0.73,1.51,15.70,XY25,NTT,AC
72,TATAMOTORS,1065.00,-55.15,11.0,X-LC,1.12,142369.0,-134154.0,244234.0,-40.64,-48.51,171.55,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
33,ICICIGI,2260.25,-20.58,41.0,X-MC,1.46,136478.0,345.0,30776.0,0.59,0.25,22.55,22.86,91.0,0.01,0.99,12.90,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,31.0,X-SC,37.83,51794.0,-13212.0,164140.0,-0.84,-20.32,316.91,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-55.15,11.0,X-LC,1.12,142369.0,-134154.0,244234.0,-40.64,-48.51,171.55,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
82,VBL,671.64,-22.00,36.0,X-LC,10.52,280848.0,-34994.0,147670.0,-0.51,-11.08,52.58,35.67,5.0,-0.24,2.04,1.24,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.42,36.0,X-MC,7.67,220750.0,-42615.0,155960.0,-0.62,-16.18,70.65,43.04,84.0,-0.27,1.60,2.48,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-15.54,33.0,X-MC,5.47,177792.0,-22714.0,119867.0,-0.29,-11.33,67.42,48.45,90.0,-0.19,1.29,2.53,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,31.0,X-SC,37.83,51794.0,-13212.0,164140.0,-0.84,-20.32,316.91,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-40.56,36.0,X-SC,3.28,81037.0,-64123.0,140048.0,-0.21,-44.17,172.82,52.31,136.0,-0.46,0.59,8.47,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.44,37.0,X-SC,4.32,94927.0,-33743.0,79036.0,-0.46,-26.22,83.26,35.20,219.0,-0.43,0.69,8.95,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.05,54.0,X-SC,1.02,108998.0,6994.0,36929.0,0.63,6.86,33.88,43.06,122.0,0.19,0.79,25.15,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.21,108525.0,-19413.0,66548.0,0.06,-15.17,61.32,36.84,143.0,-0.29,0.79,11.10,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.33,46.0,X-LC,12.25,281314.0,-64642.0,132893.0,-0.48,-18.68,47.24,19.73,1.0,-0.49,2.04,3.61,X40,ATH,IT
39,INFY,2275.00,-18.52,52.0,X-LC,7.63,318052.0,5164.0,166532.0,-1.43,1.65,52.36,54.87,3.0,0.03,2.31,8.41,X40,BTT,IT
41,ITC,452.00,-39.61,39.0,X-LC,2.75,194500.0,-5638.0,26530.0,-0.38,-2.82,13.64,10.44,4.0,-0.21,1.41,2.59,X40,NTT,FMCG
82,VBL,671.64,-22.00,36.0,X-LC,10.52,280848.0,-34994.0,147670.0,-0.51,-11.08,52.58,35.67,5.0,-0.24,2.04,1.24,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,40.0,H-LC,7.29,245376.0,-16243.0,135448.0,-0.69,-6.21,55.20,45.57,7.0,-0.12,1.78,4.41,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-43.66,50.0,L-SC,28.75,81946.0,-31603.0,71654.0,0.21,-27.83,87.44,35.27,268.0,-0.44,0.59,100.56,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,40.0,H-SC,9.92,89685.0,-8295.0,29901.0,-0.96,-8.47,33.34,22.05,152.0,-0.28,0.65,30.36,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,106.53,61.0,H-SC,11.62,130734.0,-11745.0,31272.0,-0.66,-8.24,23.92,13.70,163.0,-0.38,0.95,54.30,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,17.42,146358.0,-2862.0,147046.0,0.13,-1.92,100.47,96.62,208.0,-0.02,1.06,58.39,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,56.0,H-SC,10.52,147002.0,-32069.0,97800.0,-2.21,-17.91,66.53,36.71,135.0,-0.33,1.07,45.86,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,17.42,146358.0,-2862.0,147046.0,0.13,-1.92,100.47,96.62,208.0,-0.02,1.06,58.39,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,56.0,H-SC,10.52,147002.0,-32069.0,97800.0,-2.21,-17.91,66.53,36.71,135.0,-0.33,1.07,45.86,XR,NTT,BANKS
31,HINDZINC,730.22,38.11,71.0,M-LC,8.70,225395.0,20319.0,92254.0,2.49,9.91,40.93,54.89,52.0,0.22,1.64,36.39,X5K,ATH,METALS
63,SHALBY,327.00,1175.92,54.0,M-SC,22.96,173562.0,-7581.0,53041.0,-1.73,-4.19,30.56,25.10,235.0,-0.14,1.26,39.09,XY24,NTT,HEALTHCARE
67,STARHEALTH,761.00,15.63,62.0,H-SC,8.12,257802.0,-23321.0,156176.0,-1.21,-8.30,60.58,47.26,144.0,-0.15,1.87,38.12,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.44
1,25,44.02
2,50,75.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.51
LC    33.63
MC    21.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.68
X40      14.85
XY25     12.06
XR       10.99
X40N     10.10
AR        8.18
OX40N     7.69
X200      1.82
X5K       1.64
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.71
X-LC    22.52
X-MC    16.06
M-SC    12.69
M-LC     5.36
H-LC     4.72
X-SC     4.66
H-MC     3.80
M-MC     1.65
L-SC     1.45
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.96,-6.67,41.83
IT,13.04,-16.91,80.23
FINANCE,12.17,-12.76,63.87
MISC,6.96,-19.86,86.27
BANKS,6.47,-12.07,71.03
PAINTS,5.59,-19.81,37.67
ELECTRICAL,5.36,-11.53,50.66
INSURANCE,3.85,-4.28,43.26
AUTO,2.79,-49.22,120.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3346834.0,22
XR,1394883.0,14
AR,1200660.0,9
X40,794549.0,10
X40N,636460.0,8
OX40N,587115.0,10
XY25,521227.0,8
SR,266772.0,2
X5K,92254.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3460609.0,25
M-SC,1290575.0,15
X-LC,1137920.0,13
X-MC,1113482.0,11
X-SC,539543.0,6
H-MC,355474.0,3
H-LC,288916.0,3
M-LC,270218.0,4
L-SC,260898.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1204966.0      6
           XR         824220.0      7
           AR         821841.0      5
M-SC       XY24       695028.0      6
X-MC       XY24       593530.0      4
X-LC       X40        493178.0      5
           XY24       312092.0      2
X-SC       X40N       296367.0      4
H-SC       OX40N      272500.0      4
           SR         266772.0      2
X-MC       X40        222335.0      4
X-LC       X40N       220226.0      3
H-LC       AR         214449.0      2
H-MC       XY24       183190.0      1
X-MC       XY25       177750.0      2
L-SC       XR         171061.0      2
M-MC       XY24       165817.0      1
M-SC       AR         164370.0      2
X-SC       XY24       164140.0      1
M-SC       OX40N      151892.0      4
           XY25       147046.0      1
           XR         132239.0      2
X-MC       X40N       119867.0      1
X-LC       XY25       112424.0      3
M-LC       XR         108264.0      1
H-MC       XR          99398.0      1
M-LC       X5K         92254.0      1
L-SC       OX40N       89837.0      1
X-SC       X40         79036.0      1
H-LC       X200        74467.0      1
H-MC       OX40N       72886.0      1
H-SC       MH          70310.0      1
L-MC       XR          59701.0      1
L-LC       XY25        42378.0      1
M-LC       XY25        41629.0      1
           XY24        28071.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
